In [10]:
using Pkg
using JuMP
using HiGHS
using Ipopt
using Optimization

In [13]:
# ------------------ Parámetros ---------------------------
A = 300 * 0.25 * 0.85                # Acres utilizables
F = 15_000_000.0                     # Financiamiento máximo
t = [1000.0, 1900.0, 2700.0, 3400.0] # Impuesto por unidad
c = [50_000.0, 70_000.0, 130_000.0, 160_000.0] # Costos
a = [0.18, 0.28, 0.40, 0.50]         # Área por unidad

4-element Vector{Float64}:
 0.18
 0.28
 0.4
 0.5

## Variables continuas

In [14]:
model_lp = Model(HiGHS.Optimizer)

@variable(model_lp, x[1:4] >= 0)   # continuas

@constraint(model_lp, sum(a[i]*x[i] for i in 1:4) <= A)
@constraint(model_lp, sum(c[i]*x[i] for i in 1:4) <= F)
@constraint(model_lp, x[3] + x[4] >= 0.25 * sum(x))
@constraint(model_lp, x[1]        >= 0.20 * sum(x))
@constraint(model_lp, x[2]        >= 0.10 * sum(x))

@objective(model_lp, Max, sum(t[i]*x[i] for i in 1:4))

optimize!(model_lp)

lp_status = termination_status(model_lp)
lp_x = value.(x)
lp_Z = objective_value(model_lp)

println("===== (b) Resultado LP (continuo) =====")
println("Status LP: ", lp_status)
println("x (continuas) = ", lp_x)
println("Z_LP (impuestos) = ", lp_Z)

# Guardamos para cálculo de brecha
x_lp = copy(lp_x)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 5 rows; 4 cols; 20 nonzeros
Coefficient ranges:
  Matrix [1e-01, 2e+05]
  Cost   [1e+03, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 2e+07]
Presolving model
5 rows, 4 cols, 20 nonzeros  0s
5 rows, 4 cols, 20 nonzeros  0s
Presolve : Reductions: rows 5(-0); columns 4(-0); elements 20(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.2578114611e+02 Ph1: 4(14.302); Du: 4(125.781) 0s
          3     3.5555555556e+05 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  3.5555555556e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
===== (b) Resultado LP (continuo) =====
Status LP: OPTIMAL
x (continuas) = [37.03703703703704, 101.85185185185185, 46.29629629629629, 0.0]
Z_LP (impuestos) = 355555.

4-element Vector{Float64}:
  37.03703703703704
 101.85185185185185
  46.29629629629629
   0.0

## Variables enteras

In [15]:
model_mip = Model(HiGHS.Optimizer)

@variable(model_mip, x[1:4] >= 0, Int)  # enteras

@constraint(model_mip, sum(a[i]*x[i] for i in 1:4) <= A)
@constraint(model_mip, sum(c[i]*x[i] for i in 1:4) <= F)
@constraint(model_mip, x[3] + x[4] >= 0.25 * sum(x))
@constraint(model_mip, x[1]        >= 0.20 * sum(x))
@constraint(model_mip, x[2]        >= 0.10 * sum(x))

@objective(model_mip, Max, sum(t[i]*x[i] for i in 1:4))

optimize!(model_mip)

mip_status = termination_status(model_mip)
mip_x = value.(x)
mip_Z = objective_value(model_mip)

println("\n===== (c) Resultado MIP (entero) =====")
println("Status MIP: ", mip_status)
println("x (enteras) = ", mip_x)
println("Z_MIP (impuestos) = ", mip_Z)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 5 rows; 4 cols; 20 nonzeros; 4 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e-01, 2e+05]
  Cost   [1e+03, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 2e+07]
Presolving model
5 rows, 4 cols, 20 nonzeros  0s
5 rows, 4 cols, 20 nonzeros  0s
Objective function is integral with scale 0.01

Solving MIP model with:
   5 rows
   4 cols (0 binary, 4 integer, 0 implied int., 0 continuous, 0 domain fixed)
   20 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |   

## Comparacion

In [16]:
gap_rel = (lp_Z - mip_Z)/lp_Z
println("\n===== Comparación =====")
println("Brecha relativa = $(gap_rel*100) %")
println("Diferencia absoluta = $(lp_Z - mip_Z)")

# (Opcional) Verificación de proporciones en la solución entera
total_int = sum(mip_x)
println("\nProporciones solución entera:")
println("Sencillas  (x1): ", mip_x[1]/total_int)
println("Dobles     (x2): ", mip_x[2]/total_int)
println("Triples    (x3): ", mip_x[3]/total_int)
println("Cuádruples (x4): ", mip_x[4]/total_int)
println("Triples+Cuádruples: ", (mip_x[3]+mip_x[4])/total_int)


===== Comparación =====
Brecha relativa = 0.043749999999985446 %
Diferencia absoluta = 155.55555555550382

Proporciones solución entera:
Sencillas  (x1): 0.2000000000000008
Dobles     (x2): 0.5499999999999993
Triples    (x3): 0.172222222222265
Cuádruples (x4): 0.07777777777773498
Triples+Cuádruples: 0.25
